In [ ]:
import os
import numpy as np
import pandas as pd
import glob
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
from skimage.io import imread, imshow
from scipy.stats import kurtosis,skew
filePath = '../input/best-artworks-of-all-time/images/images'
print(cv2.__version__)
artists = pd.read_csv('../input/best-artworks-of-all-time/artists.csv')

In [ ]:
artists.shape

In [ ]:
artists.columns

In [ ]:

categories = artists.genre.unique()


In [ ]:
genre_labels = set()
for genre in categories:
    word = str(genre).split(',')
    for l in word:
        genre_labels.add(l)   
genre_labels = list(genre_labels)

In [ ]:
print(genre_labels,len(genre_labels))

In [ ]:
# image = imread(filePath+"/Alfred_Sisley"+"/Alfred_Sisley_152.jpg", as_gray=True) 
# image.shape, imshow(image)
# images = loadImages(filePath)
# def show_rgb_img(img):
#     return plt.imshow(cv2.cvtColor(img, cv2.CV_32S))

In [ ]:
# m = image.mean(axis=0).mean(axis=0)
# sd = image.std(axis=0).mean(axis=0)
# print(m,sd)
# sk = skew(skew(image))
# print(sk)
# kr = kurtosis(image,axis=None)
# print(kr)

In [ ]:
orb = cv2.ORB_create()

In [ ]:
def extractFeatures(image):
    key_points, description = orb.detectAndCompute(image, None)
    return key_points, description
# img_keypoints = cv2.drawKeypoints(image,key_points,image,flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS) # Draw circles.
# plt.figure(figsize=(16, 16))
# plt.title('ORB Interest Points')
# plt.imshow(img_keypoints); plt.show()



In [ ]:
# def image_detect_and_compute(detector, img_name):
#     """Detect and compute interest points and their descriptors."""
#     img = cv2.imread(filePath+img_name,1)
#     img = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
#     kp, des = detector.detectAndCompute(img, None)
#     return img, kp, des
    

# def draw_image_matches(detector, img1_name, img2_name, nmatches=10):
#     """Draw ORB feature matches of the given two images."""
#     img1, kp1, des1 = image_detect_and_compute(detector, img1_name)
#     img2, kp2, des2 = image_detect_and_compute(detector, img2_name)
# #     print(des1.shape,des2.shape)
#     bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
#     matches = bf.match(des1, des2)
#     matches = sorted(matches, key = lambda x: x.distance) 
    
#     img_matches = cv2.drawMatches(img1, kp1, img2, kp2, matches[:nmatches], img2, flags=2)
#     plt.figure(figsize=(16, 16))
#     plt.title(type(detector))
#     plt.imshow(img_matches); plt.show()

In [ ]:
# def getImages(path):
#     images={}
#     count=0
#     for each in glob.glob(path + "/*"):
#         word = each.split("/")[-1]
#         word = word.replace("_"," ")
#         if(artists['name'].str.contains(word).any()):
#             r = artists[artists['name'].str.contains(word)]
#             labels = str(r['genre']).split(',')
#             for l in labels:
#                 images[l]=[]
#             temp=0
#             for imagefile in glob.glob(each+"/*"):
#                 if(temp==51):
#                     break
#                 im = cv2.imread(imagefile)
#                 images[labels[0]].append(im)
#                 temp += 1
#                 count += 1
                
#     return count,images
        
# count,images = getImages(filePath)
# print(len(images['William Turner']))

In [ ]:
def getDescriptors(path):
    descriptor_list = []
    train_labels = np.array([])
    desAll = []
    for each in glob.glob(path + "/*"):
        artist = each.split("/")[-1]
        artist = artist.replace("_"," ")
        if(artists['name'].str.contains(artist).any()):
            r = artists[artists['name'].str.contains(artist)]
            labels = r.genre.unique()
            labels = labels[0].split(',')
            print(labels)
            for l in labels:
                temp=0
                for imagefile in glob.glob(each+"/*"):
                    if(temp==51):
                        break
                    im = cv2.imread(imagefile)
                    kp, ds = extractFeatures(im)
                    descriptor_list.append(ds)
                    for feature in ds:
                        desAll.append(feature)
                    temp += 1
                    train_labels = np.append(train_labels,genre_labels.index(l))
    return descriptor_list,desAll,train_labels

In [ ]:
descriptor_list,desAll, train_labels = getDescriptors(filePath)

In [ ]:
len(train_labels)

In [ ]:
print(len(descriptor_list), descriptor_list[0].shape, len(desAll),len(desAll[0]))

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters = 240)
kmeans.fit(desAll)
